In [1]:
import nltk
import numpy as np
import urllib.request
import random
import string
import pyttsx3
import speech_recognition as sr

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords 
 

### Procesamiento de Voz

In [2]:
r = sr.Recognizer()
mic = sr.Microphone()
# sr.Microphone.list_microphone_names()

### Sintesis de Voz

In [3]:
engine = pyttsx3.init()
engine.setProperty('rate',150)
engine.setProperty('voice','spanish')

In [4]:
def speak(texto):
    engine.say(texto)
    engine.runAndWait()

In [5]:
#obtener el contenido HTML de la pagina web 
res = urllib.request.urlopen("https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/que-es-el-web-scraping/")
html = res.read()
soup = BeautifulSoup(html,"html.parser")

In [6]:
#observamos la etiquetas HTML
setTag = {x.name for x in soup.find_all(True)}
print(setTag)

{'script', 'form', 'head', 'li', 'h4', 'ol', 'i', 'em', 'base', 'html', 'link', 'title', 'a', 'h2', 'meta', 'h1', 'strong', 'footer', 'style', 'span', 'label', 'noscript', 'main', 'div', 'br', 'time', 'body', 'article', 'button', 'section', 'ul', 'header', 'h3', 'picture', 'img', 'fieldset', 'source', 'nav', 'input', 'p'}


In [7]:
def searchByTag(tagName):
    Content = ''
    for tag in soup.find_all(tagName):
        if(len(tag.text) > 144):
            Content += tag.text+'\n\n'
    return Content


In [8]:
Content = searchByTag('p')
# print(Content)

In [9]:
raw = Content.lower()

In [10]:
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [11]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [12]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # print("a",tfidf[-1])
    # print("------------------")
    # print(tfidf)
    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
   

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
  
    req_tfidf = flat[-2]
    # print("------------------")
    # print(req_tfidf)
    # print(idx)
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

In [13]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

In [14]:
# flag = True
# # r = sr.Recognizer()
# # mic = sr.Microphone()
# introduces = "AGENTE: MI NOMBRE ES ROXANNE"

# print(introduces)
# speak(introduces)

# while(flag == True):

#     # user_response = input()
#     # user_response = user_response.lower() #convertimos a minuscula

    
#     # speak("Dime cual es tud duda")
#     # with sr.Microphone() as source:
#     #     audio = r.listen(source)

#     #     try:
#     #         text = r.recognize_google(audio, language='es')
#     #         entrada = format(text)
#     #         print(entrada)
#     #         #print("Lo que dijiste: {}".format(text))
#     #     except:
#     #         entrada = 'Lo siento, no te entiendo'
#     #         print(entrada)
#     #         #print("Lo siento, no te entiendo")
#     # user_response = entrada


#     speak("Dime cual es tud duda")

#     with sr.Microphone() as source:
#         r.adjust_for_ambient_noise(source)
#         audio = r.listen(source)

#     text = r.recognize_google(audio, language='es',show_all=True)
    
#     # entrada = format(text)

#     try:
#         entrada = text['alternative'][0]['transcript']
#         # print(entrada)
        
#     except:
#         entrada = ' '
#     print("user:> ",entrada)
#     user_response = entrada.lower()










#     if(user_response not in ["salir","chao","adios","bay","exit"]):

#         if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
#             flag = True
#             thanks_res = "No hay de que"
#             print("Roxanne:> "+ thanks_res)
#             speak(thanks_res)
#         else:
#             if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
#                 greets = saludos(user_response) 
#                 print("Roxanne:> " + greets)
#                 speak(greets)

#             else:#si la palabra insertada no es un saludo --> Corpus
#                 respond = response(user_response)
#                 print("Roxanne:> " + respond )
#                 speak(respond)
#                 sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
#     else:
#         flag = False
#         print("Roxanne:> Nos vemos pronto, !Gracias por su visita¡")
#         speak("Nos vemos pronto, !Gracias por su visita¡")    

In [15]:
# print(Content)

In [16]:
def speakAgent(msg):
    print(msg)
    speak(msg)


In [17]:
def chatBots():
    flag = True
    mode = True
    chat = True
    speakAgent("MI NOMBRE ES ROXANNE")
    while(mode == True):
        
        speakAgent("Si deseas hacerme preguntas por voz, por favor escriba, si, de lo contrario te respondere a las preguntas que me escribes en el formulario")
        res = input()
        
        if res.lower() in ["si","ok","y"]:
            print(res)
            chat = False
            mode = False
        elif res.lower() in ["no","not"]:
            mode = False

    while(flag == True):

        if chat:
            speakAgent("Por favor, escribame tu pregunta?")
            user_response = input()
            user_response = user_response.lower()
        else:
            speak("Dime cual es tu duda")
            with sr.Microphone() as source:
                r.adjust_for_ambient_noise(source)
                audio = r.listen(source)
                
            text = r.recognize_google(audio, language='es',show_all=True)
            try:
                entrada = text['alternative'][0]['transcript']
                # print(entrada)
            except:
                entrada = ' '
                print("user:> ",entrada)

            user_response = entrada.lower()

        if(user_response not in ["salir","chao","adios","bay","exit"]):
            if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
                flag = True
                speakAgent(thanks_res)
            else:
                if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                    greets = saludos(user_response) 
                    speakAgent(greets)

                else:#si la palabra insertada no es un saludo --> Corpus
                    respond = response(user_response)
                    speakAgent(respond)
                    sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
        else:
            flag = False
            speakAgent("Nos vemos pronto, !Gracias por su visita¡") 


In [19]:
chatBots()

MI NOMBRE ES ROXANNE
Si deseas hacerme preguntas por voz, por favor escriba, si, de lo contrario te respondere a las preguntas que me escribes en el formulario
si
el scrapingno siempre es legal.
el primer rastreador web se creó ya en 1993, cuando se presentó el primer motor de búsqueda: jumpstation.
Lo siento, no te he entendido
user:>   
Lo siento, no te he entendido
en primer lugar, los scrapers deben tener en cuenta los derechos de propiedad intelectual de los sitios web.
Nos vemos pronto, !Gracias por su visita¡


In [ ]:
# flag = True
# # r = sr.Recognizer()
# # mic = sr.Microphone()
# introduces = "AGENTE: MI NOMBRE ES ROXANNE"

# print("AGENTE:> ",introduces)
# speak(introduces)

# while(flag == True):

#     # user_response = input()
#     # user_response = user_response.lower() #convertimos a minuscula

    
#     # speak("Dime cual es tud duda")
#     # with sr.Microphone() as source:
#     #     audio = r.listen(source)

#     #     try:
#     #         text = r.recognize_google(audio, language='es')
#     #         entrada = format(text)
#     #         print(entrada)
#     #         #print("Lo que dijiste: {}".format(text))
#     #     except:
#     #         entrada = 'Lo siento, no te entiendo'
#     #         print(entrada)
#     #         #print("Lo siento, no te entiendo")
#     # user_response = entrada


#     speak("Dime cual es tu duda")

#     with sr.Microphone() as source:
#         r.adjust_for_ambient_noise(source)
#         audio = r.listen(source)

#     text = r.recognize_google(audio, language='es',show_all=True)
    
#     # entrada = format(text)

#     try:
#         entrada = text['alternative'][0]['transcript']
#         # print(entrada)
        
#     except:
#         entrada = ' '
#     print("user:> ",entrada)
#     user_response = entrada.lower()










#     if(user_response not in ["salir","chao","adios","bay","exit"]):

#         if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
#             flag = True
#             thanks_res = "No hay de que"
#             print("Roxanne:> "+ thanks_res)
#             speak(thanks_res)
#         else:
#             if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
#                 greets = saludos(user_response) 
#                 print("Roxanne:> " + greets)
#                 speak(greets)

#             else:#si la palabra insertada no es un saludo --> Corpus
#                 respond = response(user_response)
#                 print("Roxanne:> " + respond )
#                 speak(respond)
#                 sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
#     else:
#         flag = False
#         print("Roxanne:> Nos vemos pronto, !Gracias por su visita¡")
#         speak("Nos vemos pronto, !Gracias por su visita¡")  

In [ ]:
flag = True
introduces = "AGENTE: MI NOMBRE ES ROXANNE. CONTESTARE A SUS PREGUNTAS ACERCA DEL WEB SCRAPING Y SUS ASUNTOS LEGALES."

print(introduces)
speak(introduces)

while(flag == True):

    user_response = input()
    user_response = user_response.lower() #convertimos a minuscula

    if(user_response not in ["salir","chao","adios","bay","exit"]):

        if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
            flag = True
            thanks_res = "No hay de que"
            print("Roxanne:> "+ thanks_res)
            speak(thanks_res)
        else:
            if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                greets = saludos(user_response) 
                print("Roxanne:> " + greets)
                speak(greets)

            else:#si la palabra insertada no es un saludo --> Corpus
                respond = response(user_response)
                print("Roxanne:> " + respond )
                speak(respond)
                sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
    else:
        flag = False
        print("Roxanne:> Nos vemos pronto, !Gracias por su visita¡")
        speak("Nos vemos pronto, !Gracias por su visita¡")    